In [1]:
import numpy as np
import cv2 as cv
import pickle
import nbimporter
import locality_sensitive_hashing as lsh
import min_hashing as mh
import shingling as sh
import prime
import os, os.path
from os.path import join
import random

Importing Jupyter notebook from locality_sensitive_hashing.ipynb
Importing Jupyter notebook from min_hashing.ipynb
Importing Jupyter notebook from prime.ipynb
Importing Jupyter notebook from shingling.ipynb


Using TensorFlow backend.


In [2]:
###########################################     DATA CLEANING    ###########################################################

In [3]:
def delete_sobel_outliers(shingles):
    deleted = []
    for i in range(shingles.shape[0]):
        if np.count_nonzero(shingles[i]) == 0:
            deleted.append(i)
    if deleted:
        print(len(deleted))
    shingles = np.delete(shingles, (deleted), axis=0)
    return shingles

In [4]:
def cleanup(folder):
    path, dirs, files = next(os.walk(folder))
    for f in files:
        print(f)
        with open(join(folder,f), 'rb') as handle:
            shingles = pickle.load(handle)
            
        shingles = delete_sobel_outliers(shingles.T)
        
        with open(join(folder,f), 'wb') as handle:
            pickle.dump(shingles, handle, protocol=pickle.HIGHEST_PROTOCOL)
    return

In [5]:
# Cleans pickles in sobel folder
#cleanup("sobel/")

In [6]:
#########################################     GENERATING DATASET    ########################################################

In [7]:
# Shape of each shingle = (Shingle_size x Image_count)
def generate_seperately(mode):
    s = sh.Shingling()
    _, dirs, _ = next(os.walk("../data"))
    if mode == "CNN":
        for i in range(18):
            shingle = s.generate_shingles(join("../data", dirs[i]), 224, 224, "CNN", layer="block5_pool")
            with open(dirs[i] + '_cnn.pickle', 'wb') as handle:
                pickle.dump(shingle, handle, protocol=pickle.HIGHEST_PROTOCOL)
                
    elif mode == "Sobel":
        for i in range(18):
            shingle = s.generate_shingles(join("../data", dirs[i]), 224, 224, "Sobel")
            with open(dirs[i] + '_sobel.pickle', 'wb') as handle:
                pickle.dump(shingle, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [8]:
def get_cached_shingles(mode, n_selections):
    _, _, files = next(os.walk(mode))
    randIndex = random.sample(range(len(files)), n_selections)
    randIndex.sort()
    result = None
    total_pairs = []
    label_map = []
    for i in randIndex:
        print(join(mode,files[i]))
        with open(join(mode,files[i]), 'rb') as handle:
            shingles = np.transpose(pickle.load(handle))
        for e in range(np.shape(shingles)[0]):
            label_map.append(files[i][0:2])
        if result is not None:
            result = np.concatenate((shingles,result))
        else:
            result = shingles
        total_pairs.append(np.shape(shingles)[0] * (np.shape(shingles)[0]-1) / 2) 
    return result, label_map, total_pairs